#             Assignment No- 3 Web Scraping

# 1. Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [36]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
driver.minimize_window()

In [38]:
try:
    #opening a amazon page on automated chrome browser
    driver.get("http://www.amazon.in/")
    #to take input from user requred for search
    item=input("\nEnter the item to search on amazon shopping website:\n")
    #opening web page to search for user entered item 
    search_box = driver.find_element(By.ID,"twotabsearchtextbox").send_keys(item)
    #click on the search button
    driver.maximize_window()
    search_button = driver.find_element(By.ID,"nav-search-submit-text").click()

except Exception as e:
    print(e)


Enter the item to search on amazon shopping website:
gitar


# 2.In the above question, now scrape the following details of each product listed in first 3 pages of your 
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search 
results then scrape all the products available under that product name. Details to be scraped are: "Brand 
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [39]:
try:
    driver.execute_script("window.scrollTo(0,500)")
   #for scraping
    Brand = []
    Prod_name = []
    Return_exchange = []
    Expected_delivary = []
    Availability = []
    product_url = []
    Price = []
    
except NoSuchElementException as e:
    print("Exception raised:",e)
    driver.find_element(By.XPATH,"/html/body/div[1]/div[2]/div[1]/div[1]/div/span[1]/div[1]/div[3]/div/div/div/div/div/div/div[2]/div[2]/h2/a").click()
    

In [40]:
try:
    # for scraping all Product urls
    #scroll to pixel 800 of webpage
    prod_urls = []
    start=0
    end=3
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
        for i in url[0:10]:
            prod_urls.append(i.get_attribute("href"))
    nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    nxt_button.click()
    time.sleep(2)

except NoSuchElementException as e:
    print(e)
    
#print(len(prod_urls)) #all urls number we print


In [43]:
for url in prod_urls[:10]:#loop for every gitar in list
    driver.get(url)
    time.sleep(2)
    try:
            with open('Product_details_intern34_ass3_Q2.csv','w',encoding='utf8',newline='') as f:
            #writer will write in file name 'Product_details_intern34_ass3_Q2.csv' object is created for stored in to thewriter variable name
                thewriter = writer(f)
                #this will be the column name lets say header
                header=['Brand Name','Price','Return/Exchange','Expected Delivery','Availability','Product URL']
                #writerwill write the row in csv file
                thewriter.writerow(header)
                
                # scraping Product name
                product_nametags = driver.find_element(By.XPATH,'//*[@id="titleSection"]//h1//span')
                Prod_name.append(product_nametags.text)
                #thewriter.writerow(Prod_name)
                
                #Scraping availability status of of product
                availabilitytags = driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
                Availability.append(availabilitytags.text)
                #thewriter.writerow(Availability)
                
                #scraping Product Price
                pricetags = driver.find_element(By.XPATH,'//span[@class="a-price-whole"]')
                Price.append(pricetags.text)
                #thewriter.writerow(Price)
                
                # scraping expected_delivary date from 3 pages first 10 products
                expected_delivarytags = driver.find_element(By.XPATH,'//span[@class="a-text-bold"]')
                Expected_delivary.append(expected_delivarytags.text)
                #thewriter.writerow(Expected_delivary)
                
                
                #scroll down to extract brands and return/exchang
                driver.execute_script("window.scrollTo(0,900)")
        
                #scraping Brand names 
                Brandtags = driver.find_element(By.XPATH,'//*[@id="productOverview_feature_div"]//div//table//tbody//tr[1]//td[2]//span')
                Brand.append(Brandtags.text)
                #thewriter.writerow(Brand)
                
                # scraping return/exchange policy
                ReturnExchangetags =driver.find_element(By.XPATH,'//*[@id="RETURNS_POLICY"]//span//div[2]//a')
                Return_exchange.append(ReturnExchangetags.text)
                #thewriter.writerow(Return_exchange)
                
                #write the info into csv file
                infoList=[Brand,Price,Return_exchange,Expected_delivary,Availability,prod_urls]
                thewriter.writerow(infoList)
        
    except NoElementException as e:
            Brand.append('-')
            Return_exchange.append('-')
            Prod_name.append('-')
            Expected_delivary.append('-')
            Price.append('-')
            prod_urls.append('-')
   
  

In [57]:
#Read the csv file 'Product_details_ass3_2.csv' and store the read_csv object int product_data variable using pd
data=pd.read_csv('Product_details_intern34_ass3_Q2.csv')
#for print prod_data_first3pages) the csv file data :---->
for i in data:
    print("\n",data[i])


 0    ['Medellin', 'Medellin', 'Wooden', 'Multi-colo...
Name: Brand Name, dtype: object

 0    ['2,399', '2,429', '799', '899', '2,090', '2,4...
Name: Price, dtype: object

 0    ['7 days Returnable', '7 days Replacement', '7...
Name: Return/Exchange, dtype: object

 0    ['Sunday, 25 December', 'Saturday, 24 December...
Name: Expected Delivery, dtype: object

 0    ['In stock.', 'In stock.', 'In stock.', 'In st...
Name: Availability, dtype: object

 0    ['https://www.amazon.in/sspa/click?ie=UTF8&spc...
Name: Product URL, dtype: object


In [58]:
#print(len(Brand)) #Print length of Brand name 
#for i in Brand:
   # print(i)
#for i in  Return_exchange:
 #   print(i)
#print(len(Return_exchange))
#print(len(Prod_name))
#print(len(Expected_delivary))
#print(len(Price))
#print(len(Availability))
#print(len(prod_urls))

In [59]:
#creating a data frame
import pandas as pd
df2= pd.DataFrame({"Brand":Brand[0:10],"Price(₹)":Price[0:10],"Return_exchange":Return_exchange[0:10],"Expected_delivary":Expected_delivary[0:10],"Availability":Availability[0:10],"prod_urls":prod_urls[0:10]})
df2



,Brand,Price(₹),Return_exchange,Expected_delivary,Availability,prod_urls
0,Medellin,"2,399",7 days Returnable,"Sunday, 25 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Medellin,"2,429",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Wooden,799,7 days Replacement,"Monday, 26 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Multi-coloured,899,7 days Replacement,"Sunday, 25 December",In stock.,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,Juârez,"2,090",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
5,JUAREZ,"2,445",7 days Replacement,"Sunday, 25 December",In stock.,https://www.amazon.in/Acoustic-Cutaway-RDS-Str...
6,JUAREZ,"2,514",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/Juarez-Acoustic-Guitar-C...
7,BigPlayer,"9,664",7 days Replacement,"Saturday, 24 December",Only 2 left in stock.,https://www.amazon.in/BigPlayer-gitar-listrik-...
8,JUAREZ,"6,864",7 days Replacement,"Saturday, 24 December",In stock.,https://www.amazon.in/JUAREZ-Arpeggio-Guitar-J...
9,YAMAHA,"11,390",7 days Replacement,"Tuesday, 27 December",Only 1 left in stock.,https://www.amazon.in/Yamaha-Acoustic-Guitar-D...


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [1]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [2]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [3]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
driver.minimize_window()

In [4]:
try:
    #opening a amazon page on automated chrome browser
    driver.get("http://www.images.google.com/")
    driver.maximize_window()
    item=input("\nEnter the item to search on google images\n")
    #opening web page to search for user entered item 
    search_box = driver.find_element(By.XPATH,'//input[@class="gLFyf"]').send_keys(item)
    #click on the search button
    driver.maximize_window()
    search_button = driver.find_element(By.XPATH,'//button[@class="Tg7LZd"]').click()
except Exception as e:
    print(e)


Enter the item to search on google images
cars


In [7]:
import requests
for _ in range(0,20):
    driver.execute_script("window.scrollBy(0,10)")

images = driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')

img_urls = []
img_data = []
for image in images:
    source = image.get_attribute('src')
    if source is not None:
        if(source[0:4] == 'http'):
            img_urls.append(source)
i=1
for i in range(len(img_urls)):
    if i>10:
        break
    print("Downloading {0} of {1} images" .format(i,10))
    response = requests.get(img_urls[i])
    file = open(r"C:\Users\Atharv\Desktop\IMAGE\image"+str(i)+".jpg","wb")
    file.write(response.content)


# 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be 
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [183]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [190]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [191]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [192]:
try:
    #opening a flipkart page on automated chrome browser
    driver.get("http://www.flipkart.com/")
    
    #closing login popup
    driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]').click()
    
    #to take input from user need to search
    item=input("\nEnter the item to search on flipkart for a smartphone e.g.: Oneplus Nord, pixel 4A) \n")
    #opening web page to search for user entered item
    search_box = driver.find_element(By.XPATH,'//input[@class="_3704LK"]').send_keys(item)
    #click on the search button
    search_button = driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]').click() 
    driver.maximize_window()

except NoSuchException as e:
    driver.get("http://www.flipkart.com/")
    item=input("\nEnter the item to search on flipkart for a smartphone (e.g.: Oneplus Nord, pixel 4A) \n")
    search_box = driver.find_element(By.XPATH,'//input[@class="_3704LK"]').send_keys(item)
    print(e)
    


Enter the item to search on flipkart for a smartphone e.g.: Oneplus Nord, pixel 4A) 
pixel 4A


In [194]:
try:
    # for scrolling the page 
    driver.execute_script("window.scrollTo(0,100)")
    # for scraping all Product urls
    start=0
    end=1
    prod_urls = []  #list to store all product url
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
        for i in url:
            prod_urls.append(i.get_attribute("href"))
            #nxt_button = driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]'
            #nxt_button.click()
            time.sleep(2)
        #print number of items present on 1st page     
        print(len(prod_urls))
                        
                       
except Exception as e:
    print(e)
    

24


In [195]:
Modelname = []
Phonename = []
Storage_ROM = []
Storage_RAM = []
Primary_Camera = []
Secondary_Camera = []
Display_Size =[]
Battery_Capacity = []
Price = []
Color = []

#for scraping each items details
for url in prod_urls[0:24]:
    driver.get(url)
    time.sleep(5)
    try:
        with open('Smartphone_intern34_ass3_Q2.csv','w',encoding='utf8',newline='') as f:
            #writer will write in file name 'Product_details_intern34_ass3_Q2.csv' object is created for stored in to thewriter variable name
            thewriter = writer(f)
            #this will be the column name lets say header
            header=["Phonename","Price(₹)","Modelname","Color","Display_Size","Storage_ROM","Storage_RAM","Primary_Camera","Secondary_Camera","Battery","prod_urls"]
            thewriter.writerow(header)
            
            #Scroll up    
            js = 'window.scrollTo(0, document.documentElement.scrollHeight)'
            driver.execute_script(js)
            time.sleep(3)
            #scraping product name,Price
            
            Phonenametags = driver.find_element(By.XPATH,'//*[@class="yhB1nd"]//span')
            Phonename.append(Phonenametags.text[0:10])
            
            Pricetags =driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
            Price.append(Pricetags.text.replace("₹",""))
            
            #scroll down
            driver.execute_script("window.scrollTo(0,2200)")
            #to click on real More
            driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]').click()
            
            #for scraping model name
            Modelnametags = driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]//tr[3]//td[2]')
            Modelname.append(Modelnametags.text)
            
            #for scraping color
            Colortags = driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]//tr[4]//td[2]')
            Color.append(Colortags.text)
            
            #driver.execute_script("window.scrollTo(0,500)")
            
            #for scraping Display size
            displaytags = driver.find_element(By.XPATH,'//div[2][@class="_3k-BhJ"]//tr[1]//td[2]')
            Display_Size.append(displaytags.text)
            
            #scroll down
            driver.execute_script("window.scrollTo(0,1000)")
            
            #for scraping storage ROM
            ROMtags = driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]//tr[1]//td[2]')
            Storage_ROM.append(ROMtags.text)
            
            #for scraping RAM
            RAMtags = driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]//tr[2]//td[2]')
            Storage_RAM.append(RAMtags.text)
            
            # for scraping Primary camera
            rearcamera_tags = driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]//tr[2]//td[2]')
            Primary_Camera.append(rearcamera_tags.text)
            
            #for scraping Secondary camera
            frontcamera_tags = driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]//tr[5]//td[2]')
            Secondary_Camera.append(frontcamera_tags.text)
            
            #scroll down
            driver.execute_script("window.scrollTo(0,900)")
        
            #for scraping Battery Capacity
            batterytags = driver.find_element(By.XPATH,'/html/body/div[1]/div/div[3]/div[1]/div[2]/div[9]/div[5]/div/div[2]/div[1]/div[10]/table/tbody/tr/td[2]/ul/li')
            Battery_Capacity.append(batterytags.text)
            
            #Scroll up    
            js = 'window.scrollTo(0, document.documentElement.scrollHeight)'
            driver.execute_script(js)
            time.sleep(3)
            
            #write the info into csv file
            Smartphone_infoList=[Phonename,Price,Modelname,Color,Display_Size,Storage_ROM,Storage_RAM,Primary_Camera,Secondary_Camera,Battery_Capacity,prod_urls]
            thewriter.writerow(Smartphone_infoList)
            
            
    except Exception as e:
        Phonename.append('-')
        Price.append('-')
        Modelname.append('-')
        Color.append('-')
        Display_Size.append('-')
        Storage_ROM.append('-')
        Storage_RAM.append('-')
        Primary_Camera.append('-')
        Secondary_Camera.append('-')
        Battery_Capacity.append('-')
        prod_urls.append('-')

In [196]:
print(Phonename)
print(Price)
print(Modelname)
print(Color)
print(Display_Size)
print(Storage_ROM)
print(Storage_RAM)
print(Primary_Camera)
print(Secondary_Camera)
print(Battery_Capacity)

['Google Pix', '-', 'realme 10 ', 'realme 10 ', 'Redmi Note', '-', 'realme 10 ', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'realme C30', 'vivo T1 44', 'vivo T1 44', 'realme 10 ', 'realme 10 ', 'realme 10 ', 'realme 10 ', '-', 'vivo T1 44', 'vivo T1 44', 'vivo T1 44', 'realme C33', 'realme C33', 'realme C33', 'realme C33']
['31,999', '-', '19,999', '19,999', '12,489', '-', '18,999', '5,999', '5,999', '5,999', '6,999', '6,999', '6,999', '14,499', '14,499', '24,999', '25,999', '25,999', '24,999', '-', '14,499', '15,999', '15,999', '8,999', '8,999', '9,999', '9,999']
['Pixel 4a', '-', '10 Pro 5G', '10 Pro 5G', '-', '10 Pro 5G', 'C30', 'C30', 'C30', 'C30', 'C30', 'C30', 'T1 44W', 'T1 44W', '10 Pro+ 5G', '10 Pro+ 5G', '10 Pro+ 5G', '10 Pro+ 5G', '-', 'T1 44W', 'T1 44W', 'T1 44W', 'C33', 'C33', 'C33', 'C33']
['Just Black', '-', 'Hyperspace', 'Nebula Blue', '-', 'Dark Matter', 'Denim Black', 'Bamboo Green', 'Lake Blue', 'Lake Blue', 'Bamboo Green', 'Denim Black', 'S

In [197]:
#Read the csv file 'Product_details_ass3_2.csv' and store the read_csv object int product_data variable using pd
data=pd.read_csv('Smartphone_intern34_ass3_Q2.csv')
#for print prod_data_first3pages) the csv file data :---->
for i in data:
    print("\n",data[i])


 0    ['Google Pix', '-', 'realme 10 ', 'realme 10 '...
Name: Phonename, dtype: object

 0    ['31,999', '-', '19,999', '19,999', '12,489', ...
Name: Price(₹), dtype: object

 0    ['Pixel 4a', '-', '10 Pro 5G', '10 Pro 5G', '-...
Name: Modelname, dtype: object

 0    ['Just Black', '-', 'Hyperspace', 'Nebula Blue...
Name: Color, dtype: object

 0    ['14.76 cm (5.81 inch)', '-', '17.07 cm (6.72 ...
Name: Display_Size, dtype: object

 0    ['128 GB', '-', '128 GB', '128 GB', '-', '128 ...
Name: Storage_ROM, dtype: object

 0    ['6 GB', '-', '8 GB', '8 GB', '-', '6 GB', '2 ...
Name: Storage_RAM, dtype: object

 0    ['12.2MP Rear Camera', '-', '108MP + 2MP', '10...
Name: Primary_Camera, dtype: object

 0    ['8MP Front Camera', '-', '16MP Front Camera',...
Name: Secondary_Camera, dtype: object

 0    ['-', '5000 mAh', '5000 mAh', '-', '5000 mAh',...
Name: Battery, dtype: object

 0    ['https://www.flipkart.com/google-pixel-4a-jus...
Name: prod_urls, dtype: object


In [199]:
#creating a data frame
import pandas as pd
df4= pd.DataFrame({"Phonename": Phonename[0:24],"Price(₹)":Price[0:24],"Modelname":Modelname[0:24],"Color":Color[0:24],"Display_Size":Display_Size[0:24],"Storage_ROM":Storage_ROM[0:24],"Storage_RAM":Storage_RAM[0:24],"Primary_Camera":Primary_Camera[0:24],"Secondary_Camera":Secondary_Camera[0:24],"BatteryCapacity":Battery_Capacity[0:24],"prod_urls":prod_urls[0:24]})
df4


,Phonename,Price(₹),Modelname,Color,Display_Size,Storage_ROM,Storage_RAM,Primary_Camera,Secondary_Camera,BatteryCapacity,prod_urls
0,Google Pix,"31,999",Pixel 4a,Just Black,14.76 cm (5.81 inch),128 GB,6 GB,12.2MP Rear Camera,8MP Front Camera,-,https://www.flipkart.com/google-pixel-4a-just-...
1,-,-,-,-,-,-,-,-,-,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-hype...
2,realme 10,"19,999",10 Pro 5G,Hyperspace,17.07 cm (6.72 inch),128 GB,8 GB,108MP + 2MP,16MP Front Camera,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-nebu...
3,realme 10,"19,999",10 Pro 5G,Nebula Blue,17.07 cm (6.72 inch),128 GB,8 GB,108MP + 2MP,16MP Front Camera,-,https://www.flipkart.com/redmi-note-11-space-b...
4,Redmi Note,"12,489",-,-,-,-,-,-,-,5000 mAh,https://www.flipkart.com/realme-10-pro-5g-dark...
5,-,-,10 Pro 5G,Dark Matter,17.07 cm (6.72 inch),128 GB,6 GB,108MP + 2MP,16MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-denim-blac...
6,realme 10,"18,999",C30,Denim Black,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-bamboo-gre...
7,realme C30,"5,999",C30,Bamboo Green,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-lake-blue-...
8,realme C30,"5,999",C30,Lake Blue,16.51 cm (6.5 inch),32 GB,2 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-lake-blue-...
9,realme C30,"5,999",C30,Lake Blue,16.51 cm (6.5 inch),32 GB,3 GB,8MP Rear Camera,5MP Front Camera,5000 mAh,https://www.flipkart.com/realme-c30-bamboo-gre...


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.


In [224]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [225]:
#import laibraries
#import libraries
import selenium
from selenium import webdriver
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from csv import writer
from pprint import pprint

In [325]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [326]:
try:
    #opening a flipkart page on automated chrome browser
    driver.get("https://www.google.com/maps")
    #to take input from user need to search
    item=input("\nlocation a city searched on google maps:")
    #opening web page to search for user entered item
    search_box = driver.find_element(By.XPATH,'//input[@class="searchboxinput xiQnY"]').send_keys(item)
    #click on the search button
    search_button = driver.find_element(By.XPATH,'//button[@class="mL3xi"]').click() 
    driver.maximize_window()
    

except NoSuchException as e:
    print(e)


location a city searched on google maps:Pune


In [327]:
import re
lar_lng = []
lar = []
long = []
try:
    url_string = driver.current_url
    print("URL extracted:",url_string)
    lar_lng = re.findall(r'@(.*)data',url_string)
    #print(lar_lng)
    #to separate list 
    new=[]
    for i in lar_lng:
        for j in lar_lng:
            new.append(j.split(','[0]))
    #print(new)
    #to retrieve and print latitude,longitude of search city
    lar = new[0][0]
    long = new[0][1]
    print("latitude of",item,"is",lar)
    print("longitude of",item,"is",long)
except Exception as e:
    print(e)

URL extracted: https://www.google.com/maps/search/Pune/@18.4489437,73.8530825,15z/data=!3m1!4b1
latitude of Pune is 18.4489437
longitude of Pune is 73.8530825


# 6. Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) 
from trak.in.

In [77]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [257]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [258]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [259]:
try:
    
    # opening the trak page on automated chrome browser
    driver.get("https://trak.in/tags/business/2022/09/28/indian-startups-receive-rs-7900-crore-vc-funding-in-august-2022-vc-fundings-up-by-9-7/#:~:text=Indian%20Startups%20Receive%20Rs%207900,Business%20of%20Tech%2C%20Mobile%20%26%20Startups")
    driver.maximize_window()
    #close ad popup
    time.sleep(3)
    #driver.find_element(By.XPATH,'/html/body/div[1]/div[2]/div[2]/div/div/div[2]/div/div/div[3]').click()
    #click on funding deals
    
    fundstags =driver.find_element(By.XPATH,'//li[@class="menu-have-icon menu-icon-type-fontawesome menu-item menu-item-type-post_type menu-item-object-page better-anim-fade menu-item-51510"]//a')
    fundstags.click()
    #scroll down
    driver.execute_script("window.scrollTo(0,400)")

except Exception as e:
    driver.get("https://trak.in/tags/business/2022/09/28/indian-startups-receive-rs-7900-crore-vc-funding-in-august-2022-vc-fundings-up-by-9-7/#:~:text=Indian%20Startups%20Receive%20Rs%207900,Business%20of%20Tech%2C%20Mobile%20%26%20Startups")
    print(e)
    
    

In [142]:
Monthtags = driver.find_element(By.XPATH,'//h2[@class="tablepress-table-name tablepress-table-name-id-57"]')
print(Monthtags.text)
leadge_tabletags = driver.find_element(By.XPATH,'//table[@class="tablepress tablepress-id-57 dataTable no-footer"]')
header = []
header = leadge_tabletags.text.split("'',")
for i in header:
    for j in header:
        newheader.append(header.text.split(",''[0]"))

newheader

April, 2021


AttributeError: 'list' object has no attribute 'text'

In [128]:
allrowtags = driver.find_element(By.XPATH,'//tr[@class="row-1 odd"]').text
allrows= allrowtags.text
  
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//tr[@class="row-1 odd"]//tr"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x00FFF243]
	(No symbol) [0x00F87FD1]
	(No symbol) [0x00E7D04D]
	(No symbol) [0x00EAC0B0]
	(No symbol) [0x00EAC22B]
	(No symbol) [0x00EDE612]
	(No symbol) [0x00EC85D4]
	(No symbol) [0x00EDC9EB]
	(No symbol) [0x00EC8386]
	(No symbol) [0x00EA163C]
	(No symbol) [0x00EA269D]
	GetHandleVerifier [0x01299A22+2655074]
	GetHandleVerifier [0x0128CA24+2601828]
	GetHandleVerifier [0x010A8C0A+619850]
	GetHandleVerifier [0x010A7830+614768]
	(No symbol) [0x00F905FC]
	(No symbol) [0x00F95968]
	(No symbol) [0x00F95A55]
	(No symbol) [0x00FA051B]
	BaseThreadInitThunk [0x765400F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A67B8E+238]


In [ ]:
rowentry = driver.find_element(By.XPATH,'//tr[@class="row-2 even"]')
for i in rowentry:
    row

In [5]:
newheader= []
for i in header:
    for j in header:
        newheader.append(j.split(' '[0]))
print(newheader)

NameError: name 'header' is not defined

# 7. Write a program to scrap all the available details of best gaming laptops from digit.in

In [475]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [476]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [477]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()

In [478]:
try:
    
    # opening the trak page on automated chrome browser
    driver.get("https://digit.in/")
    driver.maximize_window()
    #close ad popup
    time.sleep(3)
    option=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/nav/div/input')
    option.click()
    time.sleep(3)
    g_mart=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/nav/div/ul/div/li[8]/a')
    g_mart.click()
    time.sleep(2)
    #scroll down
    driver.execute_script("window.scrollTo(0,800)")
except Exception as e:
    print(e)

In [479]:
#select the option gamimg Laptop
gam_lapu_option=driver.find_element(By.XPATH,'/html/body/div[3]/div/div[2]/ul/li[16]/span/a/img')
gam_lapu_option.click()

In [480]:
try:
    # for scraping all Product urls
    #scroll to pixel 800 of webpage
    prod_urls = []
    start=0
    end=1
    for page in range(start,end): #for loop for scrapping 3 page
        url = driver.find_elements(By.XPATH,'//div[@class="article_row"]/h2/a')
        for i in url:
            prod_urls.append(i.get_attribute("href"))
            #Scroll down  
        driver.execute_script("window.scrollTo(0,600)")
        time.sleep(2)

except Exception as e:
    print(e)

In [481]:
print((prod_urls))

['https://www.digit.in/top-products/best-asus-gaming-laptop-3912.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-rs-50-000-3715.html', 'https://www.digit.in/top-products/best-hp-gaming-laptop-3933.html', 'https://www.digit.in/top-products/best-dell-gaming-laptop-3935.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-60k-3980.html', 'https://www.digit.in/top-products/best-lenovo-gaming-laptops-in-india-3982.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-40k-3978.html', 'https://www.digit.in/top-products/best-msi-gaming-laptops-in-india-3983.html', 'https://www.digit.in/top-products/gaming-laptops-under-rs-70-000-3973.html', 'https://www.digit.in/top-products/best-gaming-laptops-under-1-lakh-3979.html']


In [483]:
for url in prod_urls[0:3]:
    driver.get(url)
    time.sleep(5)
        
    driver.execute_script("window.scrollTo(0,600)")
    #i = driver.find_element(By.XPATH,'//h3[@class="ga-not-fired"]')
    #name= i.text
    #for i in driver.find_element(By.XPATH,'//h3[@class="ga-not-fired"]'):
    name = []
    desc = []
    available = []
    nametags = driver.find_element(By.XPATH,'//h3[@class="ga-not-fired"]//a')
    name.append(nametags.text)
     
    desctags = driver.find_element(By.XPATH,'//div[@class="article_row big-image"]//p')
    desc.append(desctags.text)
     
    availabletags = driver.find_element(By.XPATH,'//div[@class="companies_des"]')
    available.append(availabletags.text)
    

In [484]:
info =[name,desc,available]
info

[['01. OMEN by HP Laptop 17-ck1023TX'],
 ["Starting things off with the OMEN by HP Laptop 17, this particular laptop is the best gaming laptop that you can get from HP bar none. If cost is no issue, then this is the best that HP can offer to a gamer. The hefty price tag of 3.09 lakhs is backed up with top-of-the-line specs, including a 12th gen Intel Core i9 processor, an NVIDIA RTX 3080Ti with 16GB of GDDR6 VRAM, 32GB of DDR5 RAM and a whopping 2TB of NVMe SSD storage. You will certainly not be wanting for more in the specs department. <p>Coming to the display, you've got a 17-inch QHD display (that's 2560x1440), with a refresh rate of 165Hz. Even at a higher resolution, with the hardware packed into the OMEN 17 you should be able to easily run most games on high to ultra settings with ease. So if you're looking for the best HP gaming laptop and budget is not an issue for you, then this is the laptop to go for.</p>"],
 ['Available\n₹ 102300\nBUY NOW']]

In [486]:
df9 = pd.DataFrame(data=info)
df9


,0
0,01. OMEN by HP Laptop 17-ck1023TX
1,Starting things off with the OMEN by HP Laptop...
2,Available\n₹ 102300\nBUY NOW


# 8. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be 
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [487]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [532]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [539]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()
# opening the trak page on automated chrome browser
driver.get("http://www.forbes.com/")
driver.maximize_window()

In [540]:
try:
    #click on menu bar
    driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[1]').click()
    #click on option  Billionaires
    option = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]')
    option.click()
    driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[2]/div[3]/ul/li[1]/a').click()
    #driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/span/svg').click()
    #option.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/span/svg/path').click()
    
except Exception as e:
    print(e)

In [546]:
driver.execute_script("window.scrollTo(0,2000)")

In [547]:
header =['Rank','Name','NetWorth','Age','Country/Territory','Source','Industry']

In [589]:
allrows = driver.find_element(By.XPATH,'//div[@class="table-row-group__container"]')
length =[]
newlength = []

length = allrows.text.split(", "[0])


length


#row = []
#row = driver.find_element(By.XPATH,'//div[@class="table-row "]')
#row1 = []
   # for eachrow in len(row):
#row.append(row.text)
 


['1.\nElon Musk\n$219 B\nUnited States\nTesla',
 " SpaceX\n2.\nJeff Bezos\n$171 B\nUnited States\nAmazon\n3.\nBernard Arnault & family\n$158 B\nFrance\nLVMH\n4.\nBill Gates\n$129 B\nUnited States\nMicrosoft\n5.\nWarren Buffett\n$118 B\nUnited States\nBerkshire Hathaway\n6.\nLarry Page\n$111 B\nUnited States\nGoogle\n7.\nSergey Brin\n$107 B\nUnited States\nGoogle\n8.\nLarry Ellison\n$106 B\nUnited States\nsoftware\nWhen people start telling you that you're crazy",
 ' you just might be on to the most important innovation in your life.\nLARRY ELLISON\nLarry Ellison is chairman',
 ' chief technology officer and cofounder of software giant Oracle',
 ' of which he owns about 35%.\nFULL PROFILE\nWEALTH HISTORY\nCLICK TO REVEAL NET WORTH BY YEAR\nNET WORTH\n$106 B\nON FORBES LISTS\n#4\n#8\n#7\n#5\n9.\nSteve Ballmer\n$91.4 B\nUnited States\nMicrosoft\n10.\nMukesh Ambani\n$90.7 B\nIndia\ndiversified\n11.\nGautam Adani\n$90 B\nIndia\ninfrastructure',
 " commodities\n12.\nMichael Bloomberg\n$82 B\

In [579]:
billdf = pd.DataFrame(data=length)
billdf.head(10)

,0
0,1.\nElon Musk\n$219 B\nUnited States\nTesla
1,SpaceX\n2.\nJeff Bezos\n$171 B\nUnited States...
2,you just might be on to the most important in...
3,chief technology officer and cofounder of sof...
4,of which he owns about 35%.\nFULL PROFILE\nWE...
5,commodities\n12.\nMichael Bloomberg\n$82 B\nU...


# 10. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, 
overall reviews, privates from price, dorms from price, facilities and property description

In [590]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [591]:
#import library
#import required libraries
import selenium
import pandas as pd
import re
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException
from selenium.webdriver.common.by import By
import time
from pprint import pprint
from selenium.webdriver.support import expected_conditions as EC

In [596]:
#first connect to the driver
driver = webdriver.Chrome(r"C:\Users\Atharv\Downloads\chromedriver_win32.zip\chromedriver.exe")
#driver.minimize_window()
# opening the trak page on automated chrome browser
driver.get("https://www.hostelworld.com/")
driver.maximize_window()

In [599]:
londontags = driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
londontags.send_keys('London')

In [601]:
londonbutton = driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]')
londonbutton.click()
#driver.find_element(By.XPATH,'//button[@id="search-button"]').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div/div/div[2]/div[2]/div/div/div[4]/div/div[2]/div/div[1]/div/div/ul/li[2]"}
  (Session info: chrome=108.0.5359.125)
Stacktrace:
Backtrace:
	(No symbol) [0x00FFF243]
	(No symbol) [0x00F87FD1]
	(No symbol) [0x00E7D04D]
	(No symbol) [0x00EAC0B0]
	(No symbol) [0x00EAC22B]
	(No symbol) [0x00EDE612]
	(No symbol) [0x00EC85D4]
	(No symbol) [0x00EDC9EB]
	(No symbol) [0x00EC8386]
	(No symbol) [0x00EA163C]
	(No symbol) [0x00EA269D]
	GetHandleVerifier [0x01299A22+2655074]
	GetHandleVerifier [0x0128CA24+2601828]
	GetHandleVerifier [0x010A8C0A+619850]
	GetHandleVerifier [0x010A7830+614768]
	(No symbol) [0x00F905FC]
	(No symbol) [0x00F95968]
	(No symbol) [0x00F95A55]
	(No symbol) [0x00FA051B]
	BaseThreadInitThunk [0x765400F9+25]
	RtlGetAppContainerNamedObjectPath [0x77A67BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77A67B8E+238]


In [602]:
driver.find_element(By.XPATH,'//button[@id="search-button"]').click()

In [613]:
alldeatailtag=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div[2]/div[4]/div')
leadge = []
leadge = alldeatailtag.text.split("\n")
leadge

['Sorted by our organic, commission-free algorithm.',
 'Featured Properties',
 "Wombat's City Hostel London",
 'Hostel - 3.6km from city centre',
 '8.9',
 'Fabulous',
 'From Rs4025',
 "St Christopher's Village",
 'Hostel - 1.8km from city centre',
 '8.0',
 'Fabulous',
 'From Rs1752',
 'All Properties',
 '1',
 '/',
 '30',
 "Wombat's City Hostel London",
 'Hostel - 3.6km from city centre Show on Map',
 'Free WiFi',
 'Follows Covid-19 sanitation guidance',
 'Perfect Location',
 'Superb Staff',
 'Fantastic Cleanliness',
 '8.9',
 'Fabulous',
 '14424 Total Reviews',
 '-20%',
 'No Privates Available',
 'Dorms From',
 'Rs5031.85 Rs4025',
 'View',
 '1',
 '/',
 '30',
 "St Christopher's Village",
 'Hostel - 1.8km from city centre Show on Map',
 'Free WiFi',
 'Follows Covid-19 sanitation guidance',
 'Perfect Location',
 'Marvellous Staff',
 'Excellent Cleanliness',
 '8.0',
 'Fabulous',
 '11776 Total Reviews',
 '-10%',
 'No Privates Available',
 'Dorms From',
 'Rs1946.52 Rs1752',
 'View',
 'Selling

In [616]:
import pandas as pd
hosteldf = pd.DataFrame(data=leadge)


In [618]:
hosteldf

,0
0,"Sorted by our organic, commission-free algorithm."
1,Featured Properties
2,Wombat's City Hostel London
3,Hostel - 3.6km from city centre
4,8.9
...,...
540,View
541,First
542,1
543,2
